In [3]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
# import time
from pprint import pprint 
from proj_module import get_country, convert_milliseconds_to_year_month_day, find_location_info

In [26]:


# create rectangular bins
bin_size = 10 # degree increments
start_lat = -90
end_lat = 90
start_long = -180
end_long = 180

min_lats = np.arange(start_lat, end_lat, bin_size)
max_lats = np.arange(start_lat + bin_size, end_lat + bin_size, bin_size)
min_longs = np.arange(start_long, end_long, bin_size)
max_longs = np.arange(start_long + bin_size, end_long + bin_size, bin_size)

# create rectangular limits array with subarrays with elements [minimum_latitude, maximum_latitude, minimum_longitude, maximum_longitude]
bins = {}
for i in range(len(min_lats)):
    for j in range(len(min_longs)):
        bins[f"{i}_{j}"] = {
            "min_lat": min_lats[i],
            "max_lat": max_lats[i],
            "min_lon": min_longs[j],
            "max_lon": max_longs[j]
        }


pprint(bins)


{'0_0': {'max_lat': -80, 'max_lon': -170, 'min_lat': -90, 'min_lon': -180},
 '0_1': {'max_lat': -80, 'max_lon': -160, 'min_lat': -90, 'min_lon': -170},
 '0_10': {'max_lat': -80, 'max_lon': -70, 'min_lat': -90, 'min_lon': -80},
 '0_11': {'max_lat': -80, 'max_lon': -60, 'min_lat': -90, 'min_lon': -70},
 '0_12': {'max_lat': -80, 'max_lon': -50, 'min_lat': -90, 'min_lon': -60},
 '0_13': {'max_lat': -80, 'max_lon': -40, 'min_lat': -90, 'min_lon': -50},
 '0_14': {'max_lat': -80, 'max_lon': -30, 'min_lat': -90, 'min_lon': -40},
 '0_15': {'max_lat': -80, 'max_lon': -20, 'min_lat': -90, 'min_lon': -30},
 '0_16': {'max_lat': -80, 'max_lon': -10, 'min_lat': -90, 'min_lon': -20},
 '0_17': {'max_lat': -80, 'max_lon': 0, 'min_lat': -90, 'min_lon': -10},
 '0_18': {'max_lat': -80, 'max_lon': 10, 'min_lat': -90, 'min_lon': 0},
 '0_19': {'max_lat': -80, 'max_lon': 20, 'min_lat': -90, 'min_lon': 10},
 '0_2': {'max_lat': -80, 'max_lon': -150, 'min_lat': -90, 'min_lon': -160},
 '0_20': {'max_lat': -80, 'ma

In [29]:
base_url = "https://earthquake.usgs.gov/fdsnws/event/1/query?"

# filters for query
min_magnitude = 4
max_depth = 100          # maximum depth of earthquake
num_results = 2000
order_by = 'magnitude'



# setup base query
query = {
    "format":       "geojson",
    "starttime":     "20230101",     # Beginning of 2023 ** testing is 1 month
    "endtime":      "20231231",
    "minmagnitude": min_magnitude,
    "maxdepth":     max_depth,
    "limit":        num_results,
    "orderby":      order_by

}

earthquakes_2023 = {
    "bin_id": [],
    "bin_centre_lat": [],
    "bin_centre_lon": [],
    "quake_lat": [],
    "quake_lon": [],
    "place": [],
    "country": [],
    "magnitude": [],
    "depth": [],
    "utm": [],
    "date": []
    
}

for bin_index, bounds in bins.items():
    query["minlatitude"] = bounds["min_lat"]
    query["maxlatitude"] = bounds["max_lat"]
    query["minlongitude"] = bounds["min_lon"]
    query["maxlongitude"] = bounds["max_lon"]

    response = requests.get(url=base_url, params=query)
    
    features = response.json()["features"]
    metadata = response.json()["metadata"]

    n_earthquakes = metadata["count"]

    
    had_earthquake = n_earthquakes > 0

    if had_earthquake:
        # print(bin_index, len(features))
        for quake in features:
            geometry = quake["geometry"]["coordinates"]
            lon = geometry[0]
            lat = geometry[1]
            depth = geometry[2]
            eq_properties = quake["properties"]
            earthquakes_2023["quake_lat"].append(lat)
            earthquakes_2023["quake_lon"].append(lon)
            earthquakes_2023["depth"].append(depth)
            earthquakes_2023["place"].append(eq_properties["place"])
            earthquakes_2023["magnitude"].append(eq_properties["mag"])
            utc = eq_properties["time"]
            earthquakes_2023["utm"].append(utc)
            date_readable = convert_milliseconds_to_year_month_day(utc)
            earthquakes_2023["date"].append(date_readable)
    

            earthquakes_2023["country"].append(get_country(lat = lat, lon = lon))

            earthquakes_2023["bin_id"].append(bin_index)
            earthquakes_2023["bin_centre_lat"].append(bounds["min_lat"] + bin_size/2)
            earthquakes_2023["bin_centre_lon"].append(bounds["min_lon"] + bin_size/2)
    print(f"-------------------Retreived earthquake info from geo-bin {bin_index}-------------------")



-------------------Retreived earthquake info from geo-bin 0_0-------------------
-------------------Retreived earthquake info from geo-bin 0_1-------------------
-------------------Retreived earthquake info from geo-bin 0_2-------------------
-------------------Retreived earthquake info from geo-bin 0_3-------------------
-------------------Retreived earthquake info from geo-bin 0_4-------------------
-------------------Retreived earthquake info from geo-bin 0_5-------------------
-------------------Retreived earthquake info from geo-bin 0_6-------------------
-------------------Retreived earthquake info from geo-bin 0_7-------------------
-------------------Retreived earthquake info from geo-bin 0_8-------------------
-------------------Retreived earthquake info from geo-bin 0_9-------------------
-------------------Retreived earthquake info from geo-bin 0_10-------------------
-------------------Retreived earthquake info from geo-bin 0_11-------------------
-------------------Retreiv

In [30]:
# convert data into a dataframe
earthquakes_df = pd.DataFrame(earthquakes_2023)
# Export the City_Data into a csv
earthquakes_df.to_csv("output_data/earthquakes_2023.csv")

In [31]:
earthquakes_df

,bin_id,bin_centre_lat,bin_centre_lon,quake_lat,quake_lon,place,country,magnitude,depth,utm,date
0,2_0,-65.0,-175.0,-65.4948,-179.6522,Pacific-Antarctic Ridge,Country not found,5.8,10.0,1703754780076,"(2023, 12, 28)"
1,2_0,-65.0,-175.0,-64.3358,-178.6842,Pacific-Antarctic Ridge,Country not found,5.0,10.0,1677365507979,"(2023, 2, 25)"
2,2_0,-65.0,-175.0,-65.8497,-175.6489,Pacific-Antarctic Ridge,Country not found,4.6,10.0,1693858456124,"(2023, 9, 4)"
3,2_1,-65.0,-165.0,-63.5703,-166.7843,Pacific-Antarctic Ridge,Country not found,5.3,10.0,1702232567842,"(2023, 12, 10)"
4,2_1,-65.0,-165.0,-62.9531,-163.7912,Pacific-Antarctic Ridge,Country not found,5.0,10.0,1673912705452,"(2023, 1, 16)"
...,...,...,...,...,...,...,...,...,...,...,...
12134,17_27,85.0,95.0,85.2105,94.9430,north of Severnaya Zemlya,Country not found,4.4,10.0,1687640179869,"(2023, 6, 24)"
12135,17_28,85.0,105.0,84.1027,108.7890,north of Severnaya Zemlya,Country not found,4.2,10.0,1673579166232,"(2023, 1, 13)"
12136,17_29,85.0,115.0,81.9981,118.7067,east of Severnaya Zemlya,Country not found,4.2,10.0,1697955157272,"(2023, 10, 22)"
12137,17_30,85.0,125.0,81.1077,120.1834,east of Severnaya Zemlya,Country not found,4.0,10.0,1688994343740,"(2023, 7, 10)"


In [32]:
import hvplot.pandas


In [36]:
map_earthquakes = earthquakes_df.hvplot.points(
    "quake_lon",
    "quake_lat",
    coastline = True,
    geo = True,
    tiles = "EsriOceanBase",
    global_extent=True,
    frame_height = 800,
    size = "magnitude",
    hover_cols = ["country", "place"],
    scale = 3,
    color = "magnitude",
    cmap='RdYlBu',
    alpha = 0.5
)
map_earthquakes

/Users/benwruck/miniconda3/envs/dev/lib/python3.10/site-packages/holoviews/core/util.py:1175: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  return pd.unique(values)
/Users/benwruck/miniconda3/envs/dev/lib/python3.10/site-packages/holoviews/core/util.py:1175: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  return pd.unique(values)
/Users/benwruck/miniconda3/envs/dev/lib/python3.10/site-packages/holoviews/core/util.py:1175: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  return pd.unique(values)
/Users/benwruck/miniconda3/envs/dev/lib/python3.10/site-packages/holoviews/core/util.py:1175: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarr

:Overlay
   .WMTS.I      :WMTS   [Longitude,Latitude]
   .Points.I    :Points   [quake_lon,quake_lat]   (magnitude,country,place)
   .Coastline.I :Feature   [Longitude,Latitude]